In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import cv2 
import glob
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt 

In [0]:
training_images = []
training_labels = []
for item in glob.iglob('/content/drive/My Drive/KDEF/training_data/*.JPG'):
    img = cv2.imread(item, cv2.COLOR_BGR2RGB)
    im = Image.open(item)
    img = cv2.resize(img, (299, 299))
    label = im.filename[47:49]
    training_labels.append(label)
    training_images.append(img)

In [1]:
testing_images = []
testing_labels = []
for item in glob.iglob('/content/drive/My Drive/KDEF/test_data/*.JPG'):
    img = cv2.imread(item, cv2.COLOR_BGR2RGB)
    im = Image.open(item)
    img = cv2.resize(img, (299, 299))
    label = im.filename[43:45]
    testing_labels.append(label)
    testing_images.append(img)

NameError: ignored

In [0]:
print(np.shape(training_images))
print(np.shape(training_labels))

(2448, 299, 299, 3)
(2448,)


In [0]:
print(np.shape(testing_images))
print(np.shape(testing_labels))

(160, 299, 299, 3)
(160,)


In [0]:
arr1 = np.unique(training_labels)
arr1.sort()

dict1 = {}
for i in range(len(arr1)):
  dict1[arr1[i]] = i
dict1

{'AF': 0, 'AN': 1, 'DI': 2, 'HA': 3, 'NE': 4, 'SA': 5, 'SU': 6}

In [0]:
training_labels = [dict1.get(n, n) for n in training_labels]

In [0]:
testing_labels = [dict1.get(n, n) for n in testing_labels]

In [0]:
from keras import applications
base_model = applications.resnet50.ResNet50(weights= 'imagenet', include_top=False, input_shape= (224, 224, 3))

In [0]:
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.6)(x)
predictions = Dense(1, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [0]:
from keras.optimizers import Adam
adam = Adam(lr = 0.003)
model.compile(optimizer= adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
import numpy as np
seed = 7
np.random.seed(seed)

from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor = 'val_acc', mode='max', patience = 20)
mc = ModelCheckpoint('best_model_02.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [0]:
history = model.fit(np.array(training_images), training_labels, validation_data=(np.array(testing_images), testing_labels), epochs = 200, 
                    batch_size = 70, callbacks=[es, mc])

Train on 2448 samples, validate on 160 samples
Epoch 1/200
2448/2448 [==============================] - 63s 26ms/step - loss: nan - acc: 0.1430 - val_loss: nan - val_acc: 0.1563

Epoch 00001: val_acc improved from -inf to 0.15625, saving model to best_model_02.h5
Epoch 2/200
2448/2448 [==============================] - 63s 26ms/step - loss: nan - acc: 0.1430 - val_loss: nan - val_acc: 0.1563

Epoch 00002: val_acc did not improve from 0.15625
Epoch 3/200
2448/2448 [==============================] - 63s 26ms/step - loss: nan - acc: 0.1430 - val_loss: nan - val_acc: 0.1563

Epoch 00003: val_acc did not improve from 0.15625
Epoch 4/200
2448/2448 [==============================] - 63s 26ms/step - loss: nan - acc: 0.1430 - val_loss: nan - val_acc: 0.1563

Epoch 00004: val_acc did not improve from 0.15625
Epoch 5/200
2448/2448 [==============================] - 63s 26ms/step - loss: nan - acc: 0.1430 - val_loss: nan - val_acc: 0.1563

Epoch 00005: val_acc did not improve from 0.15625
Epoch 6/

In [0]:
2preds = model.evaluate(testing_images, testing_labels)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

160/160 [==============================] - 1s 9ms/step
Loss = 13.398167037963868
Test Accuracy = 0.16875


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()                       
drive = GoogleDrive(gauth)

In [0]:
model.save('model_77.5_percent_acc.h5')
model_file = drive.CreateFile({'title' : 'model_77.5_percent_acc.h5'})                       
model_file.SetContentFile('model_77.5_percent_acc.h5')                       
model_file.Upload()

In [0]:
model.save('best_model.h5')
model_file = drive.CreateFile({'title' : 'best_model.h5'})                       
model_file.SetContentFile('best_model.h5')                       
model_file.Upload()

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

In [0]:
file_obj = drive.CreateFile({'id': '1NPQMP3fboe1AXQn02god0K3OZ1PesSxm'})                       
file_obj.GetContentFile('keras.h5')

In [0]:
from keras import applications
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D

m = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=(299, 299, 3), pooling=None)
x = m.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(7, activation= 'softmax')(x)
incep = Model(inputs = m.input, outputs = predictions)

In [0]:
from keras.optimizers import Adam
adam = Adam(lr = 0.0001)
incep.compile(optimizer= adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

import numpy as np
seed = 7
np.random.seed(seed)

from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor = 'val_acc', mode='max', patience = 50)
mc = ModelCheckpoint('best_model_02.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [0]:
history = incep.fit(np.array(training_images), np.array(training_labels), validation_data=(np.array(testing_images), np.array(testing_labels)), epochs = 200, 
                    batch_size = 70, callbacks=[es, mc])

Train on 2448 samples, validate on 160 samples
Epoch 1/200
2448/2448 [==============================] - 128s 52ms/step - loss: 1.5140 - acc: 0.4150 - val_loss: 0.8709 - val_acc: 0.6750

Epoch 00001: val_acc improved from -inf to 0.67500, saving model to best_model_02.h5
Epoch 2/200
2448/2448 [==============================] - 87s 36ms/step - loss: 0.4962 - acc: 0.8194 - val_loss: 0.5574 - val_acc: 0.8000

Epoch 00002: val_acc improved from 0.67500 to 0.80000, saving model to best_model_02.h5
Epoch 3/200
2448/2448 [==============================] - 87s 35ms/step - loss: 0.1650 - acc: 0.9555 - val_loss: 0.3629 - val_acc: 0.8687

Epoch 00003: val_acc improved from 0.80000 to 0.86875, saving model to best_model_02.h5
Epoch 4/200
2448/2448 [==============================] - 87s 35ms/step - loss: 0.0635 - acc: 0.9833 - val_loss: 0.2729 - val_acc: 0.9125

Epoch 00004: val_acc improved from 0.86875 to 0.91250, saving model to best_model_02.h5
Epoch 5/200
2448/2448 [============================

In [0]:
import keras 
best_model = keras.models.load_model('best_model_02.h5')

In [0]:
score, acc = best_model.evaluate(np.array(testing_images), np.array(testing_labels))
print('Test score:', score)
print('Test accuracy:', acc)

160/160 [==============================] - 6s 36ms/step
Test score: 0.27186322808265684
Test accuracy: 0.94375


In [0]:
x